# Tensors
### This script introduces how to create elemental NN tensors in TensorFlow

The following is an elemental computational graph that we are going to create and execute.
It is very similar to many of the calculations that must be made in artificial neural networks.

It is formed by a linear transformation, xW + b, followed by a non-linear activation fucntion, ReLU().
Where:
* x is a D-dimensional data point.
* W is the DxM matrix of weights.
* b is the bias M-dimensional vector
![simple_graph](../images/simple_graph.png)

## LOAD CONFIGURATION

First we start with loading TensorFlow and reseting the computational graph.

In [1]:
import tensorflow as tf #Tensorflow 1.5 import warnings https://github.com/ContinuumIO/anaconda-issues/issues/6678
from tensorflow.python.framework import ops
ops.reset_default_graph()
import random as rnd
import numpy as np

/home/parrondo/anaconda3/envs/deeptrading/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/parrondo/anaconda3/envs/deeptrading/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/parrondo/anaconda3/envs/deeptrading/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


## IMPLEMENT THE MODEL

### 1. Build a graph

* a. Graph contains parameter specifications, model architecture, optimization process, ...

* b. Somewhere between 5 and 5000 lines


These are the elements involved in our graph:

* Variables are 0-ary stateful nodes which output their current value.
* Placeholders are 0-ary nodes whose value is fed in at execution time.
* Mathematical operations:
    * MatMul: Multiply two matrix values.
    * Add: Add elementwise (with broadcasting).
    * ReLU: Activate with elementwise rectified linear function.


In [2]:
b = tf.Variable(tf.zeros((100,)), name='biases')
W=tf.Variable(tf.random_uniform((1024, 100), -1, 1), name='weights')
x = tf.placeholder(tf.float32, (1, 1024), name="x")
h_i = tf.nn.relu(tf.matmul(x, W) + b, name="h_i")

In [3]:
b

<tf.Variable 'biases:0' shape=(100,) dtype=float32_ref>

In [4]:
W

<tf.Variable 'weights:0' shape=(1024, 100) dtype=float32_ref>

In [5]:
x

<tf.Tensor 'x:0' shape=(1, 1024) dtype=float32>

### 2. Start a graph session

Launch the graph in a session.

* a session: a binding to a particular execution context
    sess.run(fetches, feeds)

* Fetches: List of graph nodes. Return the outputs of these nodes.
* Feeds: Dictionary mapping from graph nodes to concrete values. Specifies the value of each graph node given in the dictionary.


In [6]:
# Initial and Run Session
sess = tf.Session()
sess.run(tf.global_variables_initializer())
rand_array = np.random.rand(1, 1024)
sess.run(h_i, feed_dict={x: rand_array})

array([[0.0000000e+00, 8.6568556e+00, 5.1436834e+00, 1.6681578e+01,
        3.8248250e+00, 3.7400441e+00, 0.0000000e+00, 4.1308441e+00,
        0.0000000e+00, 8.6496601e+00, 0.0000000e+00, 0.0000000e+00,
        4.5185556e+00, 1.1925392e+01, 0.0000000e+00, 3.0791817e+00,
        0.0000000e+00, 1.0175586e-02, 2.7840409e+00, 0.0000000e+00,
        3.8055394e+00, 1.3759440e+01, 0.0000000e+00, 1.1995993e+01,
        0.0000000e+00, 0.0000000e+00, 2.8985724e+00, 3.4192131e+00,
        0.0000000e+00, 2.2896218e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 8.0005894e+00, 0.0000000e+00, 8.8865194e+00,
        0.0000000e+00, 0.0000000e+00, 5.2340307e+00, 0.0000000e+00,
        2.0277209e+00, 0.0000000e+00, 0.0000000e+00, 8.5369577e+00,
        0.0000000e+00, 0.0000000e+00, 4.7742062e+00, 0.0000000e+00,
        1.9366996e+00, 1.3114616e+01, 5.4199939e+00, 1.3549864e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 5.6561823e+00,
        0.0000000e+00, 6.4074202e+00, 1.4850110e

### Visualizing the Variable Creation in TensorBoard
To visualize the creation of variables in Tensorboard, we will reset the computational graph and create a global initializing operation.

Typical TensorFlow graphs can have many thousands of nodes. To simplify, variable names can be grouped and the visualization uses this information to define a hierarchy on the nodes in the graph. By default, only the top of this hierarchy is shown. Here is an example that defines three operations under the hidden name scope using `tf.name_scope`.

Grouping nodes by name scopes is important to making a legible graph. If we are building a model, name scopes give us control over the resulting visualization. The better our name scopes, the better our visualization.

In [7]:
# Reset graph
ops.reset_default_graph()

b = tf.Variable(tf.zeros((100,)), name='biases')
W=tf.Variable(tf.random_uniform((1024, 100), -1, 1), name='weights')
x = tf.placeholder(tf.float32, (1, 1024), name="x")
h_i = tf.nn.relu(tf.matmul(x, W) + b, name="h_i")


# Initial and Run Session
with tf.Session() as sess:
    writer = tf.summary.FileWriter("/tmp/tensorflow_logs", sess.graph)
    sess.run(tf.global_variables_initializer())
    rand_array = np.random.rand(1, 1024)
    sess.run(h_i, feed_dict={x: rand_array})
    writer.flush()
    writer.close()


We now run the following command in our command prompt:

`tensorboard --logdir=tmp/tensorflow_logs`

And it will tell us the URL we can navigate our browser to to see Tensorboard. The default should be:

`http://0.0.0.0:6006/`

Here is the graph.
![graph_1](../images/graph_1.png)

### Creating Tensors
TensorFlow has built in function to create tensors for use in variables.  For example, we can create a zero filled tensor of predefined shape using the `tf.zeros()` function as follows.

In [8]:
one_tensor = tf.zeros([1,50])

### 3. Fetch and feed data with Session.run

* The Compilation, optimization, etc. are involved in this step. We probably will not notice


We can evaluate tensors with calling a `run()` method on our session.

In [9]:
# Start session
sess = tf.Session()
sess.run(tf.global_variables_initializer())

sess.run(one_tensor)

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.]], dtype=float32)

TensorFlow algorithms need to know which objects are variables and which are constants.We create a variable using the TensorFlow function `tf.Variable()` as follows.

In [10]:
one_var = tf.Variable(tf.zeros([1,64]))

Note that we can not run `sess.run(one_var)`, this would result in an error.  Because TensorFlow operates with computational graphs, we have to create a variable intialization operation in order to evaluate variables. We can initialize one variable at a time by calling the variable method `one_var.initializer`.

In [11]:
sess.run(one_var.initializer)
sess.run(one_var)

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],
      dtype=float32)

It is very important to control the dimensions of our entities. This is a very sensitive point and due to the high quantity of data involved in NN calculations, that detail is of major importance. Let's first start by creating variables of specific shape by declaring our row and column size.

In [12]:
row_dim = 3
col_dim = 5

Here are variables initialized to contain all zeros or ones.

In [13]:
zero_var = tf.Variable(tf.zeros([row_dim, col_dim]))
ones_var = tf.Variable(tf.ones([row_dim, col_dim]))

Now, we can call the initializer method on our variables and run them to evaluate thier contents.

In [14]:
sess.run(zero_var.initializer)
sess.run(ones_var.initializer)
print(sess.run(zero_var))
print(sess.run(ones_var))

[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
[[1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]]


In [15]:
# Type the entity class
zero_var

<tf.Variable 'Variable_1:0' shape=(3, 5) dtype=float32_ref>

In [16]:
# Type the entity class
ones_var

<tf.Variable 'Variable_2:0' shape=(3, 5) dtype=float32_ref>

### Creating Tensors Based on Other Tensor's Shape
If the shape of a tensor depends on the shape of another tensor, then we can use the TensorFlow built-in functions `ones_like()` or `zeros_like()`.

In [17]:
other_zero_var = tf.Variable(tf.zeros_like(zero_var))
other_ones_var = tf.Variable(tf.ones_like(ones_var))

sess.run(other_zero_var.initializer)
sess.run(other_ones_var.initializer)

print(sess.run(other_zero_var))
print(sess.run(other_ones_var))

[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
[[1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]]


### Filling a Tensor with a Constant
Here is how we fill a tensor with a constant.

In [18]:
filled_var = tf.Variable(tf.fill([row_dim, col_dim], 3.14))
sess.run(filled_var.initializer)
print(sess.run(filled_var))

[[3.14 3.14 3.14 3.14 3.14]
 [3.14 3.14 3.14 3.14 3.14]
 [3.14 3.14 3.14 3.14 3.14]]


We can also create a variable from an array or list of constants.

In [19]:
# Create a variable from a constant
const_var = tf.Variable(tf.constant([3, 1, 4, 1, 5, 9, 2]))

# This can also be used to fill an array:
const_fill_array = tf.Variable(tf.constant([3,1,4,1,5,9,2,6,5,3,5,8,9,7,9], shape=[row_dim, col_dim]))

sess.run(const_var.initializer)
sess.run(const_fill_array.initializer)

print(sess.run(const_var))
print(sess.run(const_fill_array))

[3 1 4 1 5 9 2]
[[3 1 4 1 5]
 [9 2 6 5 3]
 [5 8 9 7 9]]


### Creating Tensors Based on Sequences and Ranges
We can also create tensors from sequence generation functions in TensorFlow.  The TensorFlow function `linspace()` and `range()` operate very similar to the python/numpy equivalents.

In [20]:
# Linspace in TensorFlow
linear_var = tf.Variable(tf.linspace(start=0.0, stop=1.0, num=5)) # Generates [0.,0.25,0.5,0.75, 1.] includes the end

# Range in TensorFlow
sequence_var = tf.Variable(tf.range(start=6, limit=17, delta=3)) # Generates [6, 9, 12, 15] doesn't include the end

sess.run(linear_var.initializer)
sess.run(sequence_var.initializer)

print(sess.run(linear_var))
print(sess.run(sequence_var))

[0.   0.25 0.5  0.75 1.  ]
[ 6  9 12 15]


### Random Number Tensors
We can initialize tensors that come from random numbers like following.

In [21]:
rnorm_var = tf.random_normal([row_dim, col_dim], mean=0.0, stddev=1.0)
runif_var = tf.random_uniform([row_dim, col_dim], minval=0, maxval=4)

print(sess.run(rnorm_var))
print(sess.run(runif_var))

[[ 0.96592015  1.2931361   2.022893   -2.6485598  -0.5478962 ]
 [-1.3088719   1.0624593   0.39242837 -1.1949545   1.0161059 ]
 [ 0.6917905   0.04493004  1.6400851  -0.67470294  1.1682576 ]]
[[2.6161566  0.7877779  0.34059334 1.4180698  2.2867293 ]
 [1.2943611  2.835229   0.9644685  1.1431279  1.3445225 ]
 [0.00697231 1.6355882  2.2091675  0.83723354 0.33106327]]
